In [76]:
CSV_IN = 'datos/Base datos_INTA_ascii_limpio.csv'

In [77]:
print 'Cargando datos de "%s"' % CSV_IN
df = pd.read_csv(CSV_IN)
print 'Cargado dataframe de tamaño %d x %d' % df.shape

Cargando datos de "datos/Base datos_INTA_ascii_limpio.csv"
Cargado dataframe de tamaño 423 x 40


In [78]:
# Estos son funciones para hacer cuentas claras y minimalistas

def truncate_str(s, n=15):
    s = str(s)
    if len(s) > n:
        return s[:(n-3)] + '...'
    else:
        return s

def smart_value_counts(s, top=10, normalize=True, percent=False):
    vc = s.value_counts(ascending=False, normalize=normalize)
    vc.index = [truncate_str(i) for i in vc.index]
    out = vc[:top]
    if vc.sum() > out.sum():
        other = pd.Series([vc.sum() - out.sum()], index=['[OTROS] (%d tipos)' % (len(vc) - len(out))])
        out = out.append(other)
    na = 1.0 - vc.sum()
    out = out.append(pd.Series([na], index=['[NA (%.0f%%)]' % (na * 100)]))
    if percent:
        return out.apply(lambda p: '%.2f%%' % (p*100))
    return out

smart_value_counts(df['campana'], top=5, normalize=True, percent=True)

2010-2011            17.49%
2011-2012            14.89%
2013-2014            13.00%
2008-2009            12.77%
2007-2008            12.53%
[OTROS] (3 tipos)    29.31%
[NA (0%)]             0.00%
dtype: object

In [79]:
# df.describe()

In [80]:
smart_value_counts(df['campana'])

2010-2011    0.174941
2011-2012    0.148936
2013-2014    0.130024
2008-2009    0.127660
2007-2008    0.125296
2005-2006    0.101655
2009-2010    0.101655
2006-2007    0.089835
[NA (0%)]    0.000000
dtype: float64

In [81]:
# Creamos una figura por cada columno, sumarizandolo visualmente

# Configuraction visual
COLOR_HIST = sns.color_palette()[0]
COLOR_BAR = sns.color_palette()[1]
COLOR_NA = (.8, .2, .2)
BAR_ALPHA = 0.8
RUGPLOT = True

saltados, numericos = 0, 0
for i, c in enumerate(sorted(df.columns)):
    print '[%d/%d]' % (i+1, len(df.columns)), 
    if df[c].count() == 0:
        print '**SALTANDO "%s"** (todos NA) |' % c,
        saltados += 1
        continue
    fig, ax = plt.subplots(1)
    ax.set_title(c)
    print c + '',
    s = df[c]
    try:
        s = s.astype(float)
        s.hist(ax=ax, color=COLOR_HIST, weights=np.zeros(s.count()) + 1. / len(s),
               alpha=BAR_ALPHA)
        ax.set_yticklabels(['%.0f%%' % (t*100) for t in ax.get_yticks()])
        ax.set_title(c)
        na = 1.0 - float(s.count()) / len(s)
        ax.plot([min(ax.get_xticks()), max(ax.get_xticks())], [na, na],
                color=COLOR_NA, label='NA (%.0f%%)' % (na * 100), alpha=0.8,
                linestyle='--')
        ax.legend(loc='upper right')
        if RUGPLOT:
            sns.rugplot(s, color=COLOR_HIST, alpha=1.0, ax=ax)
        print '(float)',
        numericos += 1
    except ValueError:
        s = smart_value_counts(s)
        s[::-1].plot(ax=ax, kind='barh', color=[COLOR_NA] +
                     [COLOR_BAR]*(len(s) - 1), alpha=BAR_ALPHA)
        ax.set_xticklabels(['%.0f%%' % (t*100) for t in ax.get_xticks()])
        print '(categorico)',
    print
    fig.subplots_adjust(left=0.2)
    figname = ''.join([ch for ch in c.lower().replace(' ', '_')
                       if ch.isalnum() or ch in ('_-')])
    fig.savefig('figuras/explorar_hist_%s.png' % figname)
    plt.close()

print '\n'
print 'Columnos: [%d numericos, %d categoricos, %d saltados (vacios), %d total]' % (
    numericos, len(df.columns) - numericos - saltados, saltados, len(df.columns))

[1/40] analisis_suelo_k (float)
[2/40] analisis_suelo_n (float)
[3/40] analisis_suelo_p (float)
[4/40] aporte_elemento_k_kg_ha (float)
[5/40] aporte_elemento_n_kg_ha (float)
[6/40] aporte_elemento_p_kg_ha (float)
[7/40] campana (categorico)
[8/40] cultivo_antecesor (categorico)
[9/40] densidad_siembra_kg_ha (float)
[10/40] factor_rend_densidad_pl_m2 (categorico)
[11/40] factor_rend_pctfactor (float)
[12/40] factor_rend_pcthcosecha (float)
[13/40] fenologia_cosecha_fecha (categorico)
[14/40] fenologia_dpf_fecha (categorico)
[15/40] fenologia_emerg_cosecha_dias (float)
[16/40] fenologia_emerg_dpf_dias (float)
[17/40] fenologia_emerg_flor_50pct_dias (float)
[18/40] fenologia_emergencia_70pct_fecha (categorico)
[19/40] fenologia_flor_50pct_fecha (categorico)
[20/40] fenologia_siembra_fecha (categorico)
[21/40] fert_cober_clk_cant_kg_ha (float)
[22/40] fert_cober_clk_fecha (categorico)
[23/40] fert_cober_dpf_cant_n_kg_ha (float)
[24/40] fert_cober_dpf_fecha (categorico)
[25/40] fert_cober_p

In [82]:
sumario = df.describe()
sumario

,superficie_sembrada_has,superficie_cosechada_has,densidad_siembra_kg_ha,analisis_suelo_n,analisis_suelo_p,analisis_suelo_k,aporte_elemento_n_kg_ha,aporte_elemento_p_kg_ha,aporte_elemento_k_kg_ha,fert_cober_preriego_cant_n_kg_ha,fert_cober_dpf_cant_n_kg_ha,fert_cober_clk_cant_kg_ha,fenologia_emerg_dpf_dias,riego_velocidad_dias,riego_emerg_lote_completo_dias,fenologia_emerg_flor_50pct_dias,fenologia_emerg_cosecha_dias,factor_rend_pcthcosecha,rendimiento_seco_kg_ha,factor_rend_pctfactor,rinde_con_factor_kg_ha
count,423.000000,423.000000,423.000000,372.000000,372.000000,361.000000,372.000000,372.000000,361.000000,311.00000,250.000000,114.000000,249.00000,385.000000,385.000000,423.000000,385.000000,382.000000,423.000000,327.000000,330.000000
mean,38.657352,38.653097,103.966903,10.828387,23.075827,37.780709,10.828387,23.075827,37.780709,47.16701,32.572800,54.433368,52.48996,5.529870,26.677922,90.846336,131.137662,20.229843,7006.091686,0.970690,6960.750929
std,17.409913,17.409731,14.632750,6.210559,13.491036,19.829912,6.210559,13.491036,19.829912,10.68954,10.774992,26.251850,6.01216,2.997354,7.775506,8.282485,10.872809,3.437030,1451.891081,0.090794,1651.947275
min,0.900000,0.900000,45.000000,3.700000,8.720000,2.490000,3.700000,8.720000,2.490000,0.00000,23.000000,24.900000,40.00000,0.000000,2.000000,72.000000,107.000000,12.000000,2478.160920,0.523500,2427.358621
25%,27.700000,27.700000,98.000000,7.000000,13.080000,29.880000,7.000000,13.080000,29.880000,46.00000,27.600000,39.840000,45.00000,4.000000,22.000000,84.500000,124.000000,18.000000,6010.236697,0.933000,5741.582064
50%,37.000000,37.000000,100.000000,7.500000,19.349680,35.400000,7.500000,19.349680,35.400000,46.00000,27.600000,39.840000,53.00000,5.000000,26.000000,91.000000,131.000000,20.400000,7158.106371,0.997500,7228.050798
75%,47.400000,47.400000,110.000000,20.000000,23.544000,43.160000,20.000000,23.544000,43.160000,46.00000,36.800000,80.000000,58.00000,7.000000,32.000000,97.000000,138.000000,22.700000,8067.185152,1.030500,8203.673237
max,123.000000,123.000000,162.000000,25.200000,63.000000,86.320000,25.200000,63.000000,86.320000,101.20000,69.000000,100.000000,64.00000,34.000000,54.000000,112.000000,163.000000,29.500000,10905.340622,1.101000,10981.678007
